In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import time

from itertools import product
from tqdm.notebook import tqdm
from statsmodels.stats.diagnostic import acorr_ljungbox
import pandas as pd
import numpy as np
import os
import ast

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import (
    mean_absolute_percentage_error,
    mean_absolute_error,
    mean_squared_error,
    r2_score
)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2

# REGRESIÓN

## Lectura de datos

In [2]:
df = pd.read_csv('./datasets/data_treino_dv_df_2000_2010.csv')
df.head(1)

,HORA (UTC),"VENTO, DIREï¿½ï¿½O HORARIA (gr) (ï¿½ (gr))","VENTO, VELOCIDADE HORARIA (m/s)",UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),TEMPERATURA Mï¿½XIMA NA HORA ANT. (AUT) (ï¿½C),TEMPERATURA Mï¿½NIMA NA HORA ANT. (AUT) (ï¿½C),"UMIDADE RELATIVA DO AR, HORARIA (%)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)","PRECIPITAï¿½ï¿½O TOTAL, HORï¿½RIO (mm)","VENTO, RAJADA MAXIMA (m/s)",PRESSï¿½O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSï¿½O ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)
0,12:00,0.809017,1.8,69.0,60.0,22.6,20.7,61.0,888.2,0.0,3.8,888.2,887.7


In [3]:
df.columns = ['HORA','WIND_DIR_HOR','WIND_VEL_HOR','HUM_REL_MAX_ANT','HUM_REL_MIN_ANT','TEMP_MAX_ANT','TEMP_MIN_ANT','HUM_REL_HOR','PRES_ATM_NIV','PREC_HOR','RAFAGA_VIENTO','PRES_ATM_MAX_ANT','PRES_ATM_MIN_ANT']
df.head(2)

,HORA,WIND_DIR_HOR,WIND_VEL_HOR,HUM_REL_MAX_ANT,HUM_REL_MIN_ANT,TEMP_MAX_ANT,TEMP_MIN_ANT,HUM_REL_HOR,PRES_ATM_NIV,PREC_HOR,RAFAGA_VIENTO,PRES_ATM_MAX_ANT,PRES_ATM_MIN_ANT
0,12:00,0.809017,1.8,69.0,60.0,22.6,20.7,61.0,888.2,0.0,3.8,888.2,887.7
1,13:00,0.965926,2.7,62.0,55.0,24.2,22.5,55.0,888.4,0.0,4.7,888.4,888.2


In [4]:
df.drop(columns='HORA', inplace= True)

Esta función genera lag features, es decir, que representan el valor de la variables predictoras en periodos anteriores. Lo anterior es útil para predecir el valor de la variable dependiente en un periodo futuro. El formato de salida de la función es tabular.

In [5]:
def create_shifted_lagged_features(
    df: pd.DataFrame,
    response_col: str,
    num_lags: int,
    response_shift: int = 0,
    output_path: str = None
) -> pd.DataFrame:
    """
    Genera un DataFrame con características de retardo (lag features) para modelado,
    y opcionalmente lo exporta a un archivo Excel, permitiendo desplazar la variable de respuesta.

    Parámetros:
    ----------
    df : pd.DataFrame
        DataFrame original que contiene las columnas de respuesta y predictoras.
        El índice debe estar ordenado temporalmente.

    response_col : str
        Nombre de la columna que se utilizará como variable de respuesta.

    num_lags : int
        Número de períodos de retardo a considerar para las características.

    response_shift : int, opcional (por defecto 0)
        Número de períodos para desplazar la variable de respuesta.
        - Si es positivo, desplaza la respuesta hacia el futuro (predice el futuro).
        - Si es negativo, desplaza la respuesta hacia el pasado.
        - Si es 0, no se desplaza la respuesta.

    output_path : str, opcional (por defecto None)
        Ruta completa donde se exportará el DataFrame resultante en formato Excel.
        Si es None, no se exportará el archivo.

    Retorna:
    -------
    pd.DataFrame
        DataFrame con las características de retardo y la variable de respuesta.
        Las filas con valores faltantes serán eliminadas.
    """

    if response_col not in df.columns:
        raise ValueError(f"La columna de respuesta '{response_col}' no está en el DataFrame.")

    # Determinar automáticamente las columnas predictoras
    predictor_cols = df.columns.drop(response_col)

    # Crear DataFrame para características de retardo
    lag_features = pd.DataFrame(index=df.index)

    for col in predictor_cols:
        for lag in range(1, num_lags + 1):
            lag_features[f'{col}_lag_{lag}'] = df[col].shift(lag)

    # Construir DataFrame final
    final_df = lag_features.copy()

    # Agregar variable de respuesta desplazada
    if response_shift != 0:
        final_df[response_col] = df[response_col].shift(-response_shift + 1)
    else:
        final_df[response_col] = df[response_col]

    # Eliminar valores faltantes y resetear índice
    final_df.dropna(inplace=True)
    final_df.reset_index(drop=True, inplace=True)

    # Exportar si se especifica una ruta
    if output_path:
        try:
            directory = os.path.dirname(output_path)
            if directory:
                os.makedirs(directory, exist_ok=True)
            final_df.to_excel(output_path, index=False)
            print(f"El DataFrame con características de retardo ha sido exportado exitosamente a: {output_path}")
        except Exception as e:
            print(f"Error al exportar el DataFrame a Excel: {e}")

    return final_df


## Modelos de regresión sin escalado

Con la siguiente función se evaluan los modelos de regresión sine scalado como;

- Regresión lineal
- Regresión Ridge
- Regresión Lasso
- Arbol de regresión
- Random Forest regresión
- Gradient Boosting regresión

Para cada una de las ventanas (7,14 y 21) se almacenas los resultadoos de las metricas solicitadas.

In [ ]:
def sliding_window_regression_models_lagged(
    df,
    target_col='WIND_VEL_HOR',
    T_values=[7, 14, 21],
    test_window=1,
    modelos=[
        {'name': 'lr', 'ModelClass': LinearRegression, 'params': None},
        {'name': 'ridge', 'ModelClass': Ridge, 'params': {'alpha': [0.01, 1.0, 10.0], 'fit_intercept': [True, False]}},
        {'name': 'lasso', 'ModelClass': Lasso, 'params': {'alpha': [0.01, 1.0, 10.0], 'fit_intercept': [True, False]}},
        {'name': 'tree_regressor', 'ModelClass': DecisionTreeRegressor, 'params': {'max_depth': [5, 15, 50, 100], 'min_samples_leaf': [1, 10, 20]}},
        {'name': 'tree_regressor', 'ModelClass': DecisionTreeRegressor, 'params': None},
        {'name': 'ramdon_forest', 'ModelClass': RandomForestRegressor, 'params': {'n_estimators': [100, 500], 'max_depth': [5, 10, 20], 'n_jobs': [-1]}},
        {'name': 'xgb_regressor', 'ModelClass': GradientBoostingRegressor, 'params': {'random_state': [0], 'learning_rate': [0.01], 'n_estimators': [100]}}
    ],
    save_path='./progreso'
):
    os.makedirs(save_path, exist_ok=True)
    resultados_por_T = {}

    for T in tqdm(T_values, desc="Procesando T", unit="ventana", leave=True):
        print(f"\n🧪 Ventana T = {T} días")
        T_hours = T * 24
        test_hours = test_window * 24
        total_windows = len(df) - (2 * T_hours) - test_hours + 1
        print(f'😎 Total windows {total_windows}')

        output_path = os.path.join(save_path, f'reg_resultados_T{T}.csv')
        split_dir = os.path.join(save_path, f'splits_T{T}')
        os.makedirs(split_dir, exist_ok=True)

        if os.path.exists(output_path):
            df_prev = pd.read_csv(output_path)
            if 'params' in df_prev:
                df_prev['params'] = df_prev['params'].apply(
                    lambda x: ast.literal_eval(x) if isinstance(x, str) else (None if pd.isna(x) else x)
                )
        else:
            df_prev = pd.DataFrame(columns=[
                'modelo', 'params', 'T_dias', 'T_horas', 'MAPE', 'MAE', 'RMSE', 'MSE', 'R2', 'LjungBox_p'
            ])

        # ⚡ Generar o cargar splits desde .npz
        splits = []
        for i, start in enumerate(tqdm(range(0, total_windows, 24), desc="Generando splits", leave=True)):
            split_file = os.path.join(split_dir, f'split_{i}.npz')

            if os.path.exists(split_file):
                data = np.load(split_file, allow_pickle=True)
                X_train, y_train, X_test, y_test = data['X_train'], data['y_train'], data['X_test'], data['y_test']
            else:
                data_window = df.iloc[start: start + (2 * T_hours) + test_hours].copy()

                temp_df = create_shifted_lagged_features(
                    data_window,
                    response_col=target_col,
                    num_lags=T_hours,
                    response_shift=1
                )

                train = temp_df.iloc[:-test_hours]
                test = temp_df.iloc[-test_hours:]

                X_train = train.drop(columns=[target_col]).values
                y_train = train[target_col].values
                X_test = test.drop(columns=[target_col]).values
                y_test = test[target_col].values

                np.savez_compressed(split_file,
                                    X_train=X_train,
                                    y_train=y_train,
                                    X_test=X_test,
                                    y_test=y_test)

            splits.append((X_train, y_train, X_test, y_test))

        # 🧠 Crear modelos
        models = []
        for modelo in tqdm(modelos, desc="Construyendo modelos", leave=False):
            model_name = modelo['name']
            ModelClass = modelo['ModelClass']
            param_grid = modelo['params']

            if param_grid is None:
                models.append({'name': model_name, 'model': ModelClass(), 'params': True})
            else:
                for combo in product(*param_grid.values()):
                    param_dict = dict(zip(param_grid.keys(), combo))
                    model_instance = ModelClass(**param_dict)
                    models.append({
                        'name': model_name,
                        'model': model_instance,
                        'params': param_dict
                    })

        # 🔍 Evaluar modelos
        for m in tqdm(models, desc="Evaluando modelos", leave=True):
            ya_evaluado = ((df_prev['modelo'] == m['name']) &
                           (df_prev['params'].apply(lambda p: p == m['params']))).any()

            if not df_prev.empty and ya_evaluado:
                print(f"✓ Modelo {m['name']} con {m['params']} ya evaluado para T={T}. Saltando...")
                continue

            resultados = {
                'modelo': m['name'],
                'params': m['params'],
                'T_dias': T,
                'T_horas': T_hours,
                'MAPE': [],
                'MAE': [],
                'RMSE': [],
                'MSE': [],
                'R2': [],
                'LjungBox_p': []
            }

            for X_train, y_train, X_test, y_test in tqdm(splits, desc=f"{m['name']} (T={T}d)", leave=True):
                model = m['model']
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

                residuals = y_test - y_pred

                resultados['MAPE'].append(mean_absolute_percentage_error(y_test, y_pred))
                resultados['MAE'].append(mean_absolute_error(y_test, y_pred))
                resultados['RMSE'].append(np.sqrt(mean_squared_error(y_test, y_pred)))
                resultados['MSE'].append(mean_squared_error(y_test, y_pred))
                resultados['R2'].append(r2_score(y_test, y_pred))

                if len(residuals) >= 2:
                    ljung_p = acorr_ljungbox(residuals, lags=[1], return_df=True)['lb_pvalue'].iloc[0]
                else:
                    ljung_p = np.nan

                resultados['LjungBox_p'].append(ljung_p)

            nuevo_row = pd.DataFrame([{
                'modelo': resultados['modelo'],
                'params': resultados['params'],
                'T_dias': resultados['T_dias'],
                'T_horas': resultados['T_horas'],
                'MAPE': np.mean(resultados['MAPE']),
                'MAE': np.mean(resultados['MAE']),
                'RMSE': np.mean(resultados['RMSE']),
                'MSE': np.mean(resultados['MSE']),
                'R2': np.mean(resultados['R2']),
                'LjungBox_p': np.nanmean(resultados['LjungBox_p'])
            }])

            df_prev = pd.concat([df_prev, nuevo_row], ignore_index=True)
            df_prev.to_csv(output_path, index=False)
            print(f"📦 Guardado modelo {m['name']} con {m['params']} en T={T}")

        resultados_por_T[T] = df_prev
        print(f"✔ Resultados finales guardados en: {output_path}")

    return resultados_por_T


In [ ]:
results = sliding_window_regression_models_lagged(
    df,
    target_col='WIND_VEL_HOR',
    T_values=[7,14,21],
    test_window=1,
   modelos=[
        {'name': 'lr', 'ModelClass': LinearRegression, 'params': None},
        {'name': 'ridge', 'ModelClass': Ridge, 'params': {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0], 'fit_intercept': [True, False]}},
        {'name': 'lasso', 'ModelClass': Lasso, 'params': {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0], 'fit_intercept': [True, False]}},
        {'name': 'tree_regressor', 'ModelClass': DecisionTreeRegressor, 'params': {'max_depth': [5, 10, 15, 30], 'min_samples_leaf': [1, 3, 5, 10]}},
        {'name': 'ramdon_forest', 'ModelClass': RandomForestRegressor, 'params': {'n_estimators': [100], 'max_depth': [5], 'n_jobs': [-1]}},
        {'name': 'xgb_regressor', 'ModelClass': GradientBoostingRegressor, 'params': {'random_state': [0], 'learning_rate': [0.01], 'n_estimators': [100]}}
    ],
    save_path='./progreso'
)

## Modelos de regresión con escalado

Con la siguiente función se evaluan los modelos de regresión que requieren escalado como;

- Vecinos más cercanos (KNN)
- SVR

Para cada una de las ventanas (7,14 y 21) se almacenas los resultadoos de las metricas solicitadas.

In [ ]:
def sliding_window_regression_models_lagged_scaled(
    df,
    target_col='WIND_VEL_HOR',
    T_values=[7, 14, 21],
    test_window=1,
    modelos=[
        {'name': 'Kneighboors', 'ModelClass': KNeighborsRegressor, 'params': {'n_neighbors': [5, 10,15], 'algorithm': ['ball_tree', 'kd_tree'], 'leaf_size':[30,60], 'n_jobs':[-1], 'weights': ['uniform', 'distance']}},
        {'name': 'SVR', 'ModelClass': SVR, 'params': {
          'C': [0.1, 1.0, 10],            # Regularización (más alto = menos tolerancia al error)
          'epsilon': [0.01, 0.1, 0.2],     # Margen de tolerancia al error sin penalización
          'kernel': ['rbf','linear'],          # Función de kernel (RBF = radial basis function, muy usada)
        }
      }
    ],
    save_path='./progreso'
):
    os.makedirs(save_path, exist_ok=True)
    resultados_por_T = {}

    for T in tqdm(T_values, desc="Procesando T", unit="ventana", leave=True):
        print(f"\n🧪 Ventana T = {T} días")
        T_hours = T * 24
        test_hours = test_window * 24
        total_windows = len(df) - (2 * T_hours) - test_hours + 1
        print(f'😎 Total windows {total_windows}')

        output_path = os.path.join(save_path, f's_resultados_T{T}.csv')
        split_dir = os.path.join(save_path, f'splits_T{T}')
        os.makedirs(split_dir, exist_ok=True)

        if os.path.exists(output_path):
            df_prev = pd.read_csv(output_path)
            if 'params' in df_prev:
                df_prev['params'] = df_prev['params'].apply(
                    lambda x: ast.literal_eval(x) if isinstance(x, str) else (None if pd.isna(x) else x)
                )
        else:
            df_prev = pd.DataFrame(columns=[
                'modelo', 'params', 'T_dias', 'T_horas', 'MAPE', 'MAE', 'RMSE', 'MSE', 'R2', 'LjungBox_p'
            ])

        # ⚡ Generar o cargar splits desde .npz
        splits = []
        for i, start in enumerate(tqdm(range(0, total_windows, 24), desc="Generando splits", leave=True)):
            split_file = os.path.join(split_dir, f'split_{i}.npz')

            if os.path.exists(split_file):
                data = np.load(split_file, allow_pickle=True)
                X_train, y_train, X_test, y_test = data['X_train'], data['y_train'], data['X_test'], data['y_test']
            else:
                data_window = df.iloc[start: start + (2 * T_hours) + test_hours].copy()

                temp_df = create_shifted_lagged_features(
                    data_window,
                    response_col=target_col,
                    num_lags=T_hours,
                    response_shift=1
                )

                train = temp_df.iloc[:-test_hours]
                test = temp_df.iloc[-test_hours:]

                X_train = train.drop(columns=[target_col]).values
                y_train = train[target_col].values
                X_test = test.drop(columns=[target_col]).values
                y_test = test[target_col].values

                np.savez_compressed(split_file,
                                    X_train=X_train,
                                    y_train=y_train,
                                    X_test=X_test,
                                    y_test=y_test)

            splits.append((X_train, y_train, X_test, y_test))

        # 🧠 Crear modelos
        models = []
        for modelo in tqdm(modelos, desc="Construyendo modelos", leave=False):
            model_name = modelo['name']
            ModelClass = modelo['ModelClass']
            param_grid = modelo['params']

            if param_grid is None:
                models.append({'name': model_name, 'model': ModelClass(), 'params': True})
            else:
                for combo in product(*param_grid.values()):
                    param_dict = dict(zip(param_grid.keys(), combo))
                    model_instance = ModelClass(**param_dict)
                    models.append({
                        'name': model_name,
                        'model': model_instance,
                        'params': param_dict
                    })

        # 🔍 Evaluar modelos
        for m in tqdm(models, desc="Evaluando modelos", leave=True):
            ya_evaluado = ((df_prev['modelo'] == m['name']) &
                           (df_prev['params'].apply(lambda p: p == m['params']))).any()

            if not df_prev.empty and ya_evaluado:
                print(f"✓ Modelo {m['name']} con {m['params']} ya evaluado para T={T}. Saltando...")
                continue

            resultados = {
                'modelo': m['name'],
                'params': m['params'],
                'T_dias': T,
                'T_horas': T_hours,
                'MAPE': [],
                'MAE': [],
                'RMSE': [],
                'MSE': [],
                'R2': [],
                'LjungBox_p': []
            }

            for X_train, y_train, X_test, y_test in tqdm(splits, desc=f"{m['name']} (T={T}d)", leave=True):
                model = m['model']
                #Escalo variables
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.fit_transform(X_test)
                
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

                residuals = y_test - y_pred

                resultados['MAPE'].append(mean_absolute_percentage_error(y_test, y_pred))
                resultados['MAE'].append(mean_absolute_error(y_test, y_pred))
                resultados['RMSE'].append(np.sqrt(mean_squared_error(y_test, y_pred)))
                resultados['MSE'].append(mean_squared_error(y_test, y_pred))
                resultados['R2'].append(r2_score(y_test, y_pred))

                if len(residuals) >= 2:
                    ljung_p = acorr_ljungbox(residuals, lags=[1], return_df=True)['lb_pvalue'].iloc[0]
                else:
                    ljung_p = np.nan

                resultados['LjungBox_p'].append(ljung_p)

            nuevo_row = pd.DataFrame([{
                'modelo': resultados['modelo'],
                'params': resultados['params'],
                'T_dias': resultados['T_dias'],
                'T_horas': resultados['T_horas'],
                'MAPE': np.mean(resultados['MAPE']),
                'MAE': np.mean(resultados['MAE']),
                'RMSE': np.mean(resultados['RMSE']),
                'MSE': np.mean(resultados['MSE']),
                'R2': np.mean(resultados['R2']),
                'LjungBox_p': np.nanmean(resultados['LjungBox_p'])
            }])

            df_prev = pd.concat([df_prev, nuevo_row], ignore_index=True)
            df_prev.to_csv(output_path, index=False)
            print(f"📦 Guardado modelo {m['name']} con {m['params']} en T={T}")

        resultados_por_T[T] = df_prev
        print(f"✔ Resultados finales guardados en: {output_path}")

    return resultados_por_T


In [ ]:
results = sliding_window_regression_models_lagged_scaled(
    df,
    target_col='WIND_VEL_HOR',
    T_values=[7,14,21],
    test_window=1,
    modelos=[
        {'name': 'Kneighboors', 'ModelClass': KNeighborsRegressor, 'params': {'n_neighbors': [5, 10,15], 'algorithm': ['ball_tree', 'kd_tree'], 'leaf_size':[30,60], 'n_jobs':[-1], 'weights': ['uniform', 'distance']}},
        {'name': 'SVR', 'ModelClass': SVR, 'params': {
          'C': [0.1, 1.0, 10],            # Regularización (más alto = menos tolerancia al error)
          'epsilon': [0.01, 0.1, 0.2],     # Margen de tolerancia al error sin penalización
          'kernel': ['rbf','linear'],          # Función de kernel (RBF = radial basis function, muy usada)
        }
      }
    ],
    save_path='./progreso'
)

## Redes Neuronales

### Multilayer Perceptron (MLP)

In [6]:
# Listar dispositivos disponibles
devices = tf.config.list_physical_devices('GPU')

if devices:
    print(f"TensorFlow está utilizando la GPU: {devices}")
else:
    print("TensorFlow no está utilizando la GPU")

TensorFlow está utilizando la GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
def build_mlp_model(input_dim, hidden_layers, activation='relu', learning_rate=0.001):
    model = Sequential()
    model.add(Dense(hidden_layers[0], input_dim=input_dim, activation=activation))
    for units in hidden_layers[1:]:
        model.add(Dense(units, activation=activation))
    model.add(Dense(1))  # Capa de salida (regresión)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model


def sliding_window_regression_models_lagged_MLP(
    df,
    target_col='WIND_VEL_HOR',
    T_values=[7, 14, 21],
    test_window=1,
    save_path='./progreso',
    mlp_params={
        'hidden_layers': [[32], [64]],
        'activation': ['tanh'],
        'learning_rate': [0.01],
        'epochs': [50],
        'batch_size': [32],
    },
    model_type='mlp'
):
    os.makedirs(save_path, exist_ok=True)
    resultados_por_T = {}

    for T in tqdm(T_values, desc="Procesando T", unit="ventana", leave=True):
        print(f"\n🧪 Ventana T = {T} días")
        T_hours = T * 24
        test_hours = test_window * 24
        total_windows = len(df) - (2 * T_hours) - test_hours + 1
        print(f'😎 Total windows {total_windows}')

        output_path = os.path.join(save_path, f'mlp_resultados_T{T}.csv')
        split_dir = os.path.join(save_path, f'splits_T{T}')
        os.makedirs(split_dir, exist_ok=True)

        if os.path.exists(output_path):
            df_prev = pd.read_csv(output_path)
            if 'params' in df_prev:
                df_prev['params'] = df_prev['params'].apply(
                    lambda x: ast.literal_eval(x) if isinstance(x, str) else (None if pd.isna(x) else x)
                )
        else:
            df_prev = pd.DataFrame(columns=[
                'modelo', 'params', 'T_dias', 'T_horas', 'MAPE', 'MAE', 'RMSE', 'MSE', 'R2', 'LjungBox_p'
            ])

        splits = []
        for i, start in enumerate(tqdm(range(0, total_windows, 24), desc="Generando splits", leave=True)):
            split_file = os.path.join(split_dir, f'split_{i}.npz')

            if os.path.exists(split_file):
                data = np.load(split_file, allow_pickle=True)
                X_train, y_train, X_test, y_test = data['X_train'], data['y_train'], data['X_test'], data['y_test']
            else:
                data_window = df.iloc[start: start + (2 * T_hours) + test_hours].copy()
                temp_df = create_shifted_lagged_features(
                    data_window,
                    response_col=target_col,
                    num_lags=T_hours,
                    response_shift=1
                )
                train = temp_df.iloc[:-test_hours]
                test = temp_df.iloc[-test_hours:]

                X_train = train.drop(columns=[target_col]).values
                y_train = train[target_col].values
                X_test = test.drop(columns=[target_col]).values
                y_test = test[target_col].values

                np.savez_compressed(split_file,
                                    X_train=X_train,
                                    y_train=y_train,
                                    X_test=X_test,
                                    y_test=y_test)

            splits.append((X_train, y_train, X_test, y_test))

        param_grid = list(product(*mlp_params.values()))
        param_keys = list(mlp_params.keys())

        #for combo in param_grid:
        for combo in tqdm(param_grid, desc="Evaluando combinaciones", unit="modelo", leave=True):
            param_dict = dict(zip(param_keys, combo))
            ya_realizado = not df_prev.empty and ((df_prev['modelo'] == model_type) & (df_prev['params'].apply(lambda p: p == param_dict))).any()

            if ya_realizado:
                rmse = df_prev[((df_prev['modelo'] == model_type) & (df_prev['params'].apply(lambda p: p == param_dict)))]['RMSE'].values[0]
                print(f"⏩ Saltando modelo ya evaluado: {model_type} con hiperparámetros: {param_dict} y RMSE {rmse}")
                continue

            modelo_idx = param_grid.index(combo) + 1
            total_modelos = len(param_grid)
            progreso_modelo = (modelo_idx / total_modelos) * 100
            print(f"\n🔧 Modelo {modelo_idx}/{total_modelos} ({progreso_modelo:.1f}%) - {model_type} con hiperparámetros: {param_dict}")

            resultados = {
                'modelo': model_type,
                'params': param_dict,
                'T_dias': T,
                'T_horas': T_hours,
                'MAPE': [],
                'MAE': [],
                'RMSE': [],
                'MSE': [],
                'R2': [],
                'LjungBox_p': []
            }

            #for X_train, y_train, X_test, y_test in tqdm(splits, desc=f"{model_type} {param_dict} (T={T}d)", leave=True):
            for X_train, y_train, X_test, y_test in tqdm(reversed(splits), desc=f"{model_type} {param_dict} (T={T}d)", leave=True):
                model = build_mlp_model(
                    input_dim=X_train.shape[1],
                    hidden_layers=param_dict['hidden_layers'],
                    activation=param_dict['activation'],
                    learning_rate=param_dict['learning_rate']
                )

                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.fit_transform(X_test)  # ⚠️ intencional

                early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
                model.fit(X_train, y_train, epochs=param_dict['epochs'], batch_size=param_dict['batch_size'], verbose=0, callbacks=[early_stop])
                y_pred = model.predict(X_test)

                residuals = y_test - y_pred.flatten()

                resultados['MAPE'].append(mean_absolute_percentage_error(y_test, y_pred))
                resultados['MAE'].append(mean_absolute_error(y_test, y_pred))
                resultados['RMSE'].append(np.sqrt(mean_squared_error(y_test, y_pred)))
                resultados['MSE'].append(mean_squared_error(y_test, y_pred))
                resultados['R2'].append(r2_score(y_test, y_pred))

                if len(residuals) >= 2:
                    ljung_p = acorr_ljungbox(residuals, lags=[1], return_df=True)['lb_pvalue'].iloc[0]
                else:
                    ljung_p = np.nan

                resultados['LjungBox_p'].append(ljung_p)

            nuevo_row = pd.DataFrame([{
                'modelo': resultados['modelo'],
                'params': resultados['params'],
                'T_dias': resultados['T_dias'],
                'T_horas': resultados['T_horas'],
                'MAPE': np.mean(resultados['MAPE']),
                'MAE': np.mean(resultados['MAE']),
                'RMSE': np.mean(resultados['RMSE']),
                'MSE': np.mean(resultados['MSE']),
                'R2': np.mean(resultados['R2']),
                'LjungBox_p': np.nanmean(resultados['LjungBox_p'])
            }])

            df_prev = pd.concat([df_prev, nuevo_row], ignore_index=True)
            df_prev.to_csv(output_path, index=False)
            print(f"📦 Guardado modelo {model_type} {param_dict} en T={T}")

        resultados_por_T[T] = df_prev
        print(f"✔ Resultados finales guardados en: {output_path}")

    return resultados_por_T


In [8]:
df = df.iloc[:24*21*6]
df.shape

(3024, 12)

In [ ]:
results = sliding_window_regression_models_lagged_MLP(
    df,
    target_col='WIND_VEL_HOR',
    T_values=[7,14,21],
    test_window=1,
    save_path='./progreso',
    mlp_params={
        'hidden_layers': [[16,32],[16],[32],[64],[32,64],[16,32,64]],
        'activation': ['tanh', 'linear', 'relu'],
        'learning_rate': [0.01],
        'epochs': [50],
        'batch_size': [32],
    },
    model_type='mlp'
)

Procesando T:   0%|          | 0/3 [00:00<?, ?ventana/s]


🧪 Ventana T = 7 días
😎 Total windows 2665


Generando splits:   0%|          | 0/112 [00:00<?, ?it/s]

Evaluando combinaciones:   0%|          | 0/18 [00:00<?, ?modelo/s]


🔧 Modelo 1/18 (5.6%) - mlp con hiperparámetros: {'hidden_layers': [16, 32], 'activation': 'tanh', 'learning_rate': 0.01, 'epochs': 50, 'batch_size': 32}


...

1/1 [==============================] - 0s 48ms/step
📦 Guardado modelo mlp {'hidden_layers': [32, 64], 'activation': 'relu', 'learning_rate': 0.01, 'epochs': 50, 'batch_size': 32} en T=14

🔧 Modelo 16/18 (88.9%) - mlp con hiperparámetros: {'hidden_layers': [16, 32, 64], 'activation': 'tanh', 'learning_rate': 0.01, 'epochs': 50, 'batch_size': 32}


mlp {'hidden_layers': [16, 32, 64], 'activation': 'tanh', 'learning_rate': 0.01, 'epochs': 50, 'batch_size': 3…

1/1 [==============================] - 0s 64ms/step
📦 Guardado modelo mlp {'hidden_layers': [16, 32, 64], 'activation': 'tanh', 'learning_rate': 0.01, 'epochs': 50, 'batch_size': 32} en T=14

🔧 Modelo 17/18 (94.4%) - mlp con hiperparámetros: {'hidden_layers': [16, 32, 64], 'activation': 'linear', 'learning_rate': 0.01, 'epochs': 50, 'batch_size': 32}


mlp {'hidden_layers': [16, 32, 64], 'activation': 'linear', 'learning_rate': 0.01, 'epochs': 50, 'batch_size':…

1/1 [==============================] - 0s 64ms/step


In [ ]:
df_mlp_7 = pd.read_csv('./progreso/mlp_resultados_T7.csv')
df_mlp_7.sort_values(by='RMSE', ascending= True).head(10)

### LSTM

In [ ]:
# 🧠 Genera secuencias de entrenamiento para RNN y LSTM
def create_sequences(df, target_col, window_size):
    X, y = [], []
    for i in range(len(df) - window_size):
        seq = df.iloc[i:i+window_size]
        X.append(seq.drop(columns=[target_col]).values)
        y.append(df.iloc[i+window_size][target_col])
    return np.array(X), np.array(y)

# 🧠 Construye un modelo LSTM para dependencias temporales largas
def build_lstm_model(timesteps, input_dim, hidden_units, activation='tanh', learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(hidden_units, activation=activation, input_shape=(timesteps, input_dim)))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

In [ ]:
def sliding_window_lstm_only(
    df,
    target_col='WIND_VEL_HOR',
    T_values=[7, 14,21],
    test_window=1,
    lstm_params={
        'hidden_units': [32, 64],
        'activation': ['tanh'],
        'learning_rate': [0.001],
        'epochs': [50],
        'batch_size': [32]
    },
    save_path='./progreso'
):
    os.makedirs(save_path, exist_ok=True)
    resultados_por_T = {}

    log_tiempos_path = os.path.join(save_path, 'tiempos_ejecucion.csv')
    if not os.path.exists(log_tiempos_path):
        with open(log_tiempos_path, 'w') as f:
            f.write('T_dias,modelo,parametros,duracion_segundos\n')

    for T in tqdm(T_values, desc="Procesando ventanas T"):
        print(f"\n🧭 Iniciando evaluación para ventana T={T} días...")
        inicio_ventana = time.time()
        T_hours = T * 24
        test_hours = test_window * 24
        extra = 1 * 24  # Datos adicionales necesarios para generar las secuencias del test
        total_windows = len(df) - T_hours - test_hours - extra + 1
        output_path = os.path.join(save_path, f'LSTM_resultados_T{T}.csv')

        if os.path.exists(output_path):
            df_prev = pd.read_csv(output_path)
            if 'params' in df_prev:
                df_prev['params'] = df_prev['params'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        else:
            df_prev = pd.DataFrame(columns=['modelo', 'params', 'T_dias', 'T_horas', 'MAPE', 'MAE', 'RMSE', 'MSE', 'R2', 'LjungBox_p'])

        param_grid = list(product(*lstm_params.values()))
        param_keys = list(lstm_params.keys())
        model_type = 'LSTM'

        for combo in param_grid:
            param_dict = dict(zip(param_keys, combo))
            ya_realizado = not df_prev.empty and ((df_prev['modelo'] == model_type) & (df_prev['params'].apply(lambda p: p == param_dict))).any()
            if ya_realizado:
                rmse = df_prev[((df_prev['modelo'] == model_type) & (df_prev['params'].apply(lambda p: p == param_dict)))]['RMSE'].values[0]
                print(f"⏩ Saltando modelo ya evaluado: {model_type} con hiperparámetros: {param_dict} y RMSE {rmse}")
                continue

            modelo_idx = param_grid.index(combo) + 1
            total_modelos = len(param_grid)
            progreso_modelo = (modelo_idx / total_modelos) * 100
            print(f"\n🔧 Modelo {modelo_idx}/{total_modelos} ({progreso_modelo:.1f}%) - {model_type} con hiperparámetros: {param_dict}")

            inicio_modelo = time.time()
            resultados = {k: [] for k in ['MAPE', 'MAE', 'RMSE', 'MSE', 'R2', 'LjungBox_p']}

            for start in tqdm(range(0, total_windows, 24), desc=f"     ↪ Subventanas ({model_type})", leave=False):
                # 🪟 Segmento de datos: entrenamiento + prueba + extra para completar secuencia
                data_window = df.iloc[start: start + T_hours + test_hours + extra].copy()
                train_data = data_window.iloc[:T_hours]
                test_data = data_window.iloc[T_hours:]

                # ⚙️ Escalado
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(train_data.drop(columns=[target_col]))
                X_test_scaled = scaler.fit_transform(test_data.drop(columns=[target_col]))  # ⚠️ intencionalmente fit_transform en ambos

                train_scaled = pd.DataFrame(X_train_scaled, columns=train_data.columns.drop(target_col))
                train_scaled[target_col] = train_data[target_col].values

                test_scaled = pd.DataFrame(X_test_scaled, columns=test_data.columns.drop(target_col))
                test_scaled[target_col] = test_data[target_col].values

                # 📊 Unión de train y test escalados para crear secuencias
                scaled = pd.concat([train_scaled, test_scaled])

                # 🧠 Creación de secuencias tipo LSTM
                X, y = create_sequences(scaled, target_col, T_hours)

                # 🏁 División explícita del test: últimos 24 pasos (1 día) como predicción
                X_train, y_train = X[:-24], y[:-24]
                X_test, y_test = X[-24:], y[-24:]

                # 🧱 Modelo LSTM
                model = build_lstm_model(
                    timesteps=X_train.shape[1],
                    input_dim=X_train.shape[2],
                    hidden_units=param_dict['hidden_units'],
                    activation=param_dict['activation'],
                    learning_rate=param_dict['learning_rate']
                )

                early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
                model.fit(X_train, y_train, epochs=param_dict['epochs'], batch_size=param_dict['batch_size'], verbose=0, callbacks=[early_stop])
                y_pred = model.predict(X_test).flatten()

                # 📈 Métricas
                residuals = y_test - y_pred
                resultados['MAPE'].append(mean_absolute_percentage_error(y_test, y_pred))
                resultados['MAE'].append(mean_absolute_error(y_test, y_pred))
                resultados['RMSE'].append(np.sqrt(mean_squared_error(y_test, y_pred)))
                resultados['MSE'].append(mean_squared_error(y_test, y_pred))
                resultados['R2'].append(r2_score(y_test, y_pred))
                ljung_p = acorr_ljungbox(residuals, lags=[1], return_df=True)['lb_pvalue'].iloc[0] if len(residuals) >= 2 else np.nan
                resultados['LjungBox_p'].append(ljung_p)

            # 🗃️ Guardar resultados
            nuevo_row = pd.DataFrame([{
                'modelo': model_type,
                'params': param_dict,
                'T_dias': T,
                'T_horas': T_hours,
                'MAPE': np.mean(resultados['MAPE']),
                'MAE': np.mean(resultados['MAE']),
                'RMSE': np.mean(resultados['RMSE']),
                'MSE': np.mean(resultados['MSE']),
                'R2': np.mean(resultados['R2']),
                'LjungBox_p': np.nanmean(resultados['LjungBox_p'])
            }])
            df_prev = pd.concat([df_prev, nuevo_row], ignore_index=True)
            df_prev.to_csv(output_path, index=False)

            # ⏱️ Duración por modelo
            duracion = time.time() - inicio_modelo
            with open(log_tiempos_path, 'a') as f:
                f.write(f'{T},{model_type},"{param_dict}",{duracion:.2f}\n')

            print(f"✅ Finalizado {model_type} con RMSE promedio: {np.mean(resultados['RMSE']):.4f} en {duracion:.2f} segundos")

        resultados_por_T[T] = df_prev
        duracion_ventana = time.time() - inicio_ventana
        print(f"🕒 Tiempo total para T={T} días: {duracion_ventana:.2f} segundos")

    return resultados_por_T


## Resultados

In [ ]:

def plot_rmse_bar(df, title='RMSE por Modelo', sort_ascending=True, figsize=(12, 6)):
    """
    Genera una gráfica de barras del RMSE por modelo, ordenada según el valor del RMSE.

    Parámetros:
    - df: DataFrame que debe contener las columnas 'modelo', 'params' y 'RMSE'.
    - title: Título de la gráfica.
    - sort_ascending: Booleano, si True ordena de menor a mayor RMSE.
    - figsize: Tamaño de la figura (tupla).

    Retorna:
    - None (muestra la gráfica).
    """
    df_sorted = df.sort_values(by="RMSE", ascending=sort_ascending)
    etiquetas = df_sorted['modelo'].astype(str) + ' ' + df_sorted['params'].astype(str)

    plt.figure(figsize=figsize)
    plt.bar(etiquetas, df_sorted['RMSE'])
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('RMSE')
    plt.title(title)
    plt.tight_layout()
    plt.show()

Leer resultados de los modelos evaluados.

In [ ]:
## Sin escalar
df_ns_7 = pd.read_csv('./progreso/reg_resultados_T7.csv')
df_ns_14 = pd.read_csv('./progreso/reg_resultados_T14.csv')
df_ns_21 = pd.read_csv('./progreso/reg_resultados_T21.csv')

##Escalados
df_s_7 = pd.read_csv('./progreso/s_resultados_T7.csv')
df_s_14 = pd.read_csv('./progreso/s_resultados_T14.csv')
df_s_21 = pd.read_csv('./progreso/s_resultados_T21.csv')

## RNN
df_mlp_7 = pd.read_csv('./progreso/mlp_resultados_T7.csv')
df_mlp_14 = pd.read_csv('./progreso/mlp_resultados_T14.csv')
df_mlp_21 = pd.read_csv('./progreso/mlp_resultados_T21.csv')


df_w_7 = pd.concat([df_ns_7, df_s_7, df_mlp_7])
df_w_14 = pd.concat([df_ns_14, df_s_14, df_mlp_14])
df_w_21 = pd.concat([df_ns_21, df_s_21,df_mlp_21])


In [ ]:
df_mlp_7.sort_values('RMSE', ascending=True).head(1)

In [ ]:
df_w_7.sort_values('RMSE', ascending=True).head(50)

In [ ]:
# Ver los DataFrames en memoria
plot_rmse_bar(df_w_7, title='RMSE por Modelo - T=7 días (ordenado de menor a mayor)', sort_ascending=True)
